<a href="https://colab.research.google.com/github/harryguiacorn/colab/blob/main/Ichimoku_chart_display.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step by step

In [ ]:
!pip install yfinance pandas plotly

## Download raw data and load into a data frame



In [ ]:
import yfinance as yf
import pandas as pd
import datetime

# Define the ticker symbol (SPY) and end date
ticker_symbol = "SPY"
look_back_month = 8
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=look_back_month*30)  # x months of data

# Download historical data using yfinance
data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Format the dates as strings
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

# Save the data to a CSV file with dates in the filename
csv_filename = f"{ticker_symbol}_{start_date_str}-{end_date_str}_daily_data"
csv_filename_full = f"{csv_filename}.csv"

print(f"Data for {ticker_symbol} saved to {csv_filename_full}")
data.to_csv(csv_filename_full)

In [ ]:

# Load the CSV data
data = pd.read_csv(csv_filename_full)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)


## Use Tapy to generate Ichimoku data

In [ ]:
!pip install tapy
import pandas as pd
from tapy import Indicators
from datetime import datetime

# Apply Ichimoku analysis
# ichimoku = ta.Ichimoku()
# data = ichimoku(data)

__df_ichimoku = Indicators(data)
__df_ichimoku.ichimoku_kinko_hyo()  # column_name_kijun_sen="K Line"

# Save the data with Ichimoku analysis to a new CSV file
output_csv_filename = f"{csv_filename}_ichimoku_tapy"
output_csv_filename_full = f"{output_csv_filename}.csv"
data_ichimoku_tapy = __df_ichimoku.df
data_ichimoku_tapy.to_csv(output_csv_filename_full)
# print(data_ichimoku_tapy.head)
print(f"Ichimoku analysis for SPY data saved to {output_csv_filename_full}")


Ichimoku analysis for SPY data saved to SPY_2023-03-05-2023-10-31_daily_data_ichimoku_tapy.csv


## Use Finta to generate Ichimoku data

In [ ]:
!pip install finta

from finta import TA

# Finta works but doesn't produce future senkou a or b.
# df = pd.read_csv(csv_filename_full)
output_csv_filename = f"{csv_filename}_ichimoku_finta"
output_csv_filename_full = f"{output_csv_filename}.csv"
data_ichimoku_finta = TA.ICHIMOKU(data).to_csv(output_csv_filename_full)

## Plot on a chart

## Line Chart

In [ ]:
import plotly.express as px

# Create an interactive chart
fig = px.line(data_ichimoku_tapy, x=data.index, y='Close', title='SPY Daily Closing Prices')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Close Price')
fig.show()

## Interactive Chart

In [ ]:
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio


# data["tenkan_sen"] = data_ichimoku_tapy["tenkan_sen"]
# data["kijun_sen"] = data_ichimoku_tapy["kijun_sen"]
# data["senkou_span_a"] = data_ichimoku_tapy["senkou_span_a"]
# data["senkou_span_b"] = data_ichimoku_tapy["senkou_span_b"]

# Create the interactive Ichimoku cloud chart
fig = go.Figure()

# Add candlestick chart
fig.add_trace(
    go.Candlestick(
        x=data_ichimoku_tapy.index,
        open=data_ichimoku_tapy["Open"],
        high=data_ichimoku_tapy["High"],
        low=data_ichimoku_tapy["Low"],
        close=data_ichimoku_tapy["Close"],
        name="Candlesticks",
    )
)

# Add the Tenkan-sen and Kijun-sen lines
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["tenkan_sen"],
        mode="lines",
        name="Tenkan-sen",
        line=dict(color="blue"),
    )
)
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["kijun_sen"],
        mode="lines",
        name="Kijun-sen",
        line=dict(color="red"),
    )
)

# Add Senkou Span A and Senkou Span B as filled areas
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["senkou_span_a"],
        fillcolor="rgba(0,255,0,0.2)",
        line=dict(width=2),
        name="Senkou Span A",
    )
)
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["senkou_span_b"],
        fill="tonexty",
        fillcolor="rgba(255,0,0,0.2)",
        line=dict(width=2),
        name="Senkou Span B",
    )
)  # fill area between trace0 and trace1

# Update layout
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
    title=f"{ticker_symbol} Ichimoku Cloud Chart",
    xaxis_rangeslider_visible=False
)

# Filter out empty data points on the x-axis
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
fig.update_yaxes(fixedrange=False)
# fig.update_yaxes(autorange=True)  # Set the y-axis to auto-range
# fig.update_layout(yaxis_range=[-3,3])

# Show the interactive chart
fig.show()

# Save the chart as an HTML file
fig.write_html(f"{ticker_symbol}_{start_date_str}-{end_date_str}_ichimoku_chart.html")

# Save the chart as a JPEG image
file_name = f"{ticker_symbol}_{start_date_str}-{end_date_str}_ichimoku_chart.jpg"
fig.write_image(file_name, width = 1500, scale = 1)




# Encapulate into a class

In [ ]:
!pip install -U yfinance pandas plotly tapy kaleido

In [11]:
import yfinance as yf
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.io as pio
from tapy import Indicators

class IchimokuAnalyzer:
    def __init__(self, ticker_symbol, look_back_month=8):
        self.ticker_symbol = ticker_symbol
        self.look_back_month = look_back_month
        self.end_date = datetime.datetime.now() + datetime.timedelta(days=1)  # Adjusted to include today
        self.start_date = self.end_date - datetime.timedelta(days=look_back_month * 30)
        self.csv_filename = f"{self.ticker_symbol}_{self.start_date.strftime('%Y-%m-%d')}_{self.end_date.strftime('%Y-%m-%d')}_daily_data.csv"

    def download_data(self):
        data = yf.download(self.ticker_symbol, start=self.start_date, end=self.end_date)
        data.to_csv(self.csv_filename)
        return data

    def apply_ichimoku_analysis(self):
        data = pd.read_csv(self.csv_filename)
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)

        df_ichimoku = Indicators(data)
        df_ichimoku.ichimoku_kinko_hyo()
        self.data_ichimoku_tapy = df_ichimoku.df

    def create_ichimoku_chart(self):
        fig = go.Figure()
        fig.add_trace(
            go.Candlestick(
                x=self.data_ichimoku_tapy.index,
                open=self.data_ichimoku_tapy["Open"],
                high=self.data_ichimoku_tapy["High"],
                low=self.data_ichimoku_tapy["Low"],
                close=self.data_ichimoku_tapy["Close"],
                name="Candlesticks",
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["tenkan_sen"],
                mode="lines",
                name="Tenkan-sen",
                line=dict(color="blue"),
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["kijun_sen"],
                mode="lines",
                name="Kijun-sen",
                line=dict(color="red"),
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["senkou_span_a"],
                fillcolor="rgba(0,255,0,0.2)",
                line=dict(width=2),
                name="Senkou Span A",
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["senkou_span_b"],
                fill="tonexty",
                fillcolor="rgba(255,0,0,0.2)",
                line=dict(width=2),
                name="Senkou Span B",
            )
        )
        fig.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list(
                        [
                            dict(count=1, label="1m", step="month", stepmode="backward"),
                            dict(count=6, label="6m", step="month", stepmode="backward"),
                            dict(count=1, label="YTD", step="year", stepmode="todate"),
                            dict(count=1, label="1y", step="year", stepmode="backward"),
                            dict(step="all"),
                        ]
                    )
                ),
                rangeslider=dict(visible=True),
                type="date",
            ),
            title=f"{self.ticker_symbol} Ichimoku Cloud Chart",
            xaxis_rangeslider_visible=False
        )
        fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
        fig.update_yaxes(fixedrange=False)
        self.fig = fig

    def show_chart(self):
        self.fig.show()

    def save_chart_as_html(self):
        self.fig.write_html(f"{self.ticker_symbol}_{self.start_date.strftime('%Y-%m-%d')}_{self.end_date.strftime('%Y-%m-%d')}_ichimoku_chart.html")

    def save_chart_as_image(self, file_name, width=1500, scale=1):
        self.fig.write_image(file_name, width=width, scale=scale)

# Usage example:
if __name__ == "__main__":
    analyzer = IchimokuAnalyzer("SPY", look_back_month=8)
    analyzer.download_data()
    analyzer.apply_ichimoku_analysis()
    analyzer.create_ichimoku_chart()
    analyzer.show_chart()
    analyzer.save_chart_as_html()
    analyzer.save_chart_as_image("SPY_chart.jpg")


[*********************100%%**********************]  1 of 1 completed
